# KoBERT 모델 Fine Tunning을 위한 면접 데이터 전처리


In [1]:
import glob
import json
import os
from pprint import pprint

import pandas as pd
from tqdm import tqdm

In [2]:
USER_HOME = os.path.expanduser("~")
DATA_HOME = os.path.join(
    USER_HOME, "jupyter/129.채용면접 인터뷰 데이터/01-1.정식개방데이터"
)
DATA_TRAINING = os.path.join(DATA_HOME, "Training")
DATA_VALIDATION = os.path.join(DATA_HOME, "Validation")

os.listdir(DATA_HOME)

['Validation', 'Training']

In [3]:
for d in [DATA_TRAINING, DATA_VALIDATION]:
    zip_list = glob.glob(os.path.join(d, "02.라벨링데이터/*.zip"))

    for src in tqdm(zip_list):
        dest = src.replace(".zip", "")

        if os.path.exists(dest):
            continue

        try:
            os.system(f"unzip -q '{src}' -d '{dest}' > /dev/null 2>&1")
        except Exception as e:
            print(e)

100%|██████████| 28/28 [00:00<00:00, 107349.65it/s]


In [4]:
flist = {}
for d in os.listdir(DATA_HOME):
    flist[d] = glob.glob(os.path.join(DATA_HOME, d, "*/*/*.json"))

print(flist.keys())

dict_keys(['Validation', 'Training'])


In [5]:
flist = glob.glob(os.path.join(DATA_HOME, "*/*/*/*.json"))
print(len(flist))
pprint(flist[:3])

9059
['/home/ssafy/jupyter/129.채용면접 인터뷰 '
 '데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_05.ICT_Male_Experienced/ckmk_d_ict_m_e_19325.json',
 '/home/ssafy/jupyter/129.채용면접 인터뷰 '
 '데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_05.ICT_Male_Experienced/ckmk_d_ict_m_e_19281.json',
 '/home/ssafy/jupyter/129.채용면접 인터뷰 '
 '데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_05.ICT_Male_Experienced/ckmk_d_ict_m_e_19285.json']


### 데이터 어떻게 생겼는지 확인


In [6]:
for fname in tqdm(flist[:3]):
    with open(fname, "r") as f:
        data = json.load(f)["dataSet"]["answer"]
        pprint(data)

100%|██████████| 3/3 [00:00<00:00, 1149.33it/s]

{'emotion': [],
 'intent': [{'category': 'etc',
             'expression': 'c_sincere_job',
             'text': '그리고   그 꿈을 이루기 위해서는 이 회사가 저에게는 가장 적합한 곳이라고 생각했기에 지원을 하게 '
                     '됐고 여기까지 오지 않았나 생각을 합니다.'}],
 'raw': {'text': '살아오면서 단계적인 목표를 설정을 하고 이루고 달성하고 또 실패를 하고 하면서 계속 그러한 과정이 반복이 '
                 '되는 것 같은 같습니다. 그래서 어 이전 직장이 떠나오는 시점은 그 전 단계에 세웠던 어떤 목표가 어떤 '
                 '달성된 시점이었다는 것을 말씀드리고 싶고요. 이전 직장을 떠나온 것은 이전 직장 마지막 단계에 세웠던 목표가 '
                 '이전 직장보다는 다른 직장에서 수행하는 데 더 효과적이고 자아 실현의 관점에 있어서는 더 이게 적합하다고 '
                 '생각했기 때문에 이직을 결심 했던 것 같습니다. 그리고 그 꿈을 이루기 위해서는 이 회사가 저에게는 가장 '
                 '적합한 곳이라고 생각했기에 지원을 하게 됐고 여기까지 오지 않았나 생각을 합니다. 따라서 이전 직장은 저의 '
                 '뭐 길고 기나긴 어떤 인생의 여러 가지 단계적 목표를 수행하는 하나의 과정이었다고 말씀드리고 싶구요. 물론 '
                 '여기 회사에서도 제 꿈을 위해서 제 목표를 위해서 열심히 노력을 할 거고 더 나은 목표를 위해서 또는 또 '
                 '다른 목표를 달성하기 위해서 제가 다른 근무 환경이 필요하다면 똑같은 선택을 하지 않을까 생각을 합니다.',
         'wordCount': 141},
 'summary': {'text': '이전 직장이 떠나오는 시점

In [7]:
emotion_answers = {
    "Training": [],
    "Validation": [],
}
intent_answers = {
    "Training": [],
    "Validation": [],
}

for fname in tqdm(flist, total=len(flist)):
    kind = fname.replace(DATA_HOME + "/", "").split("/")[0]

    with open(fname, "r") as f:
        try:
            data = json.load(f)["dataSet"]["answer"]

            # [text, category, expression]
            for e in data["emotion"]:
                if e["text"] == "":
                    continue
                emotion_answers[kind].append(
                    [e["text"], e["category"], e["expression"]]
                )

            for e in data["intent"]:
                if e["text"] == "":
                    continue
                intent_answers[kind].append([e["text"], e["category"], e["expression"]])
        except Exception as e:
            print(e)

 28%|██▊       | 2524/9059 [00:00<00:00, 25234.91it/s]

Invalid control character at: line 23 column 100 (char 688)
Invalid control character at: line 34 column 165 (char 1573)


100%|██████████| 9059/9059 [00:00<00:00, 24184.66it/s]


In [8]:
emotion_answers_df = {}
for k, v in emotion_answers.items():
    emotion_answers_df[k] = pd.DataFrame(
        v, columns=["text", "category", "expression"])

emotion_answers_df

{'Training':                                                   text  category  expression
 0                    라는 그런 생각이 좀 저에게 부담으로 다가오는 것 같습니다.  negative  n-distress
 1         어 이전 직장에서 새 프로젝트 제품에 대해 글로벌 허가를 진행한 적이 있습니다.   neutral      u-fact
 2    제가 담당한 제품은 의료기기였기 때문에   어 시장에 런칭하기 위해서는 반드시 각 ...   neutral      u-fact
 3    그리고 제가 업무를 추진할    때도 프로세스에 대한 이해가 좀 빨라지기 때문에 그...  positive  p-interest
 4    그리고 명상을 통해서 생각을 정리한다거나 제 마음을 차분히 하는 시간을 갖는 것을 ...  positive  p-interest
 ..                                                 ...       ...         ...
 239  어 매년 내년에는 무얼 해 봐야지 어떤 걸 계획해서 어떤 걸 한번 해봐야지 도전해 ...  negative  n-distress
 240   예를 들어서 저는 재학 대학 재학 시절에 축제를 진행한 경우에 경험이 경험이 있습니다.   neutral      u-fact
 241                     하지만 저는 개인적으로 동물 실험을 반대하는 편입니다.   neutral    u-belief
 242  또한 오락 오로지 인간의 이익을 위해 무고한 동물들을 희생시키는 것 또한 저는 비윤...   neutral    u-belief
 243  그리고 저는 어 대학원에서 공부를 할 때 어 기후융합 통합 연구소에서 일을 하면서 ...   neutral      u-fact
 
 [244 rows x 3 columns],
 'Validation':           

In [9]:
intent_answers_df = {}
for k, v in intent_answers.items():
    intent_answers_df[k] = pd.DataFrame(
        v, columns=["text", "category", "expression"])

intent_answers_df

{'Training':                                                   text     category  \
 0    어 그 리더십을 사용해서 또 어 관련자들을 또 독려하고 그 목표를 향해서 그 목표를...  personality   
 1    그래서 저는 다양한 사람들과 문제에 대해서 함께 고민하고 새로운 아이디어 또 해결할...  personality   
 2                                저는 제가 팀장 성향이라고 생각합니다.   background   
 3    실제로 프로젝트를 진행할 때 팀워크를 더 중요하고 우리가 함께 머리를 맞대서 좋은 ...  personality   
 4         어 이전 직장에서 새 프로젝트 제품에 대해 글로벌 허가를 진행한 적이 있습니다.   background   
 ..                                                 ...          ...   
 463  또한 오락 오로지 인간의 이익을 위해 무고한 동물들을 희생시키는 것 또한 저는 비윤...   background   
 464  그리고 저는 어 대학원에서 공부를 할 때 어 기후융합 통합 연구소에서 일을 하면서 ...   background   
 465  저는 그러한 갈등의 교정을 겪으면서 너무 힘들다고 생각했고 그러한 갈등을 극복하기 ...  personality   
 466  따라서 오해가 발생했다면 그 오해가 어떤 원인으로부터 발생했는지 오해의 원인을 찾고...  personality   
 467  따라서 그 입장 차이를 서로 알고 서로를 이해하면 극복해 나갈 수 있는 문제라고 생...  personality   
 
      expression  
 0         c_cop  
 1         c_cop  
 2      c_person  
 3         c_cop  
 4     c_private  
 ..     

In [10]:
emotion_answers_df["Training"].groupby(by=["category", "expression"]).size()

category  expression   
negative  n-anxiety          5
          n-distress        16
neutral   u-belief          21
          u-fact           100
positive  n-sadness          5
          p-achievement     19
          p-affection       12
          p-gratitude        1
          p-happiness       16
          p-interest        49
dtype: int64

In [11]:
intent_answers_df["Training"].groupby(by=["category", "expression"]).size()

category     expression   
attitude     m_critical       18
             m_direct          3
             m_holistic        6
background   c_person         74
             c_private        87
             c_value          16
etc          c_sincere_co      7
             c_sincere_job    13
personality  c_adp            17
             c_chl             6
             c_confl_mg       49
             c_cop            69
             c_imp            16
technology   m_acct           17
             m_admin          30
             m_sche           40
dtype: int64

In [12]:
for k, v in emotion_answers_df.items():
    v["class"] = v[["category", "expression"]].agg("-".join, axis=1)
    v = v.drop(columns=["category", "expression"])
    v.to_csv(f"./data/emotion_answers_{k}.tsv", sep="\t", index=False, encoding="utf8")

In [13]:
for k, v in intent_answers_df.items():
    v["class"] = v[["category", "expression"]].agg("-".join, axis=1)
    v = v.drop(columns=["category", "expression"])
    v.to_csv(f"./data/intent_answers_{k}.tsv", sep="\t", index=False, encoding="utf8")

In [14]:
read_test_df = pd.read_csv(
    "./data/emotion_answers_Training.tsv", sep="\t", encoding="utf8"
)
read_test_df[:3]

,text,class
0,라는 그런 생각이 좀 저에게 부담으로 다가오는 것 같습니다.,negative-n-distress
1,어 이전 직장에서 새 프로젝트 제품에 대해 글로벌 허가를 진행한 적이 있습니다.,neutral-u-fact
2,제가 담당한 제품은 의료기기였기 때문에 어 시장에 런칭하기 위해서는 반드시 각 ...,neutral-u-fact
